Create a simple agent with a simple graph. The workflow call the agent and answer to the questions

In [0]:
%pip install -U -qqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

### Import Libraries

In [0]:
from databricks_langchain import ChatDatabricks

from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

from IPython.display import Image, display
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

### Define Models

In [0]:
# define chat model with an llm
chat_model = ChatDatabricks(endpoint="databricks-claude-3-7-sonnet")

### Define Graph

In [0]:
# Define chat Node
def chat_model_node(state: MessagesState):
    return {"messages": chat_model.invoke(state["messages"])}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

### Call  Agent

In [0]:
messages = [AIMessage(f"So you said you were researching ocean mammals?", name="Bot")]
messages.append(HumanMessage(f"Yes, I know about whales. But what others should I learn about?", name="Lance"))

output = graph.invoke({'messages': messages})
for m in output['messages']:
    m.pretty_print()